# Closure

In [1]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2


from IPython.core import debugger
ist = debugger.set_trace

In [2]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [3]:
import sys
sys.path.insert(0,'../')

## Base Closure

In [14]:
%%run_and_write ../typyPRISM/closure/Closure.py
#!python
from __future__ import division,print_function
import numpy as np
class Closure:
    '''Baseclass for all closures 
    
    .. warning::
    
        Currently, this class doesn't do anything besides group all of the
        closures under a single inheritance heirarchy. This will likely
        change as needs arise.
    '''
        

Overwriting ../typyPRISM/closure/Closure.py


In [15]:
%%run_and_write ../typyPRISM/closure/AtomicClosure.py
#!python
from __future__ import division,print_function
import numpy as np
class AtomicClosure:
    '''Baseclass for all atomic closures
    
    .. warning::
    
        Currently, this class doesn't do anything besides group all of the
        *atomic* closures under a single inheritance heirarchy. This will 
        likely change as needs arise.
    '''
        

Overwriting ../typyPRISM/closure/AtomicClosure.py


In [16]:
%%run_and_write ../typyPRISM/closure/MolecularClosure.py
#!python
from __future__ import division,print_function
import numpy as np
class MolecularClosure:
    '''Baseclass for all *molecular* closures
    
    .. warning::
    
        Currently, this class doesn't do anything besides group all of the
        *molecular* closures under a single inheritance heirarchy. This will 
        likely change as needs arise.
    '''
        

Overwriting ../typyPRISM/closure/MolecularClosure.py


## PercusYevick

In [17]:
%%run_and_write ../typyPRISM/closure/PercusYevick.py
#!python
from __future__ import division,print_function
from typyPRISM.closure.AtomicClosure import AtomicClosure
import numpy as np
class PercusYevick(AtomicClosure):
    '''Percus Yevick closure written in terms of a change of variables
    
    .. note::
    
        The change of variables is necessary in order to use potentials with
        hard cores. Written in the standard form, this closure diverges with
        divergent potentials, which makes it impossible to numerically solve. 
    
    .. math::
        
        c_{i,j}(r) = (exp(-U_{i,j}(r)) - 1.0) * (1.0 + \gamma_{i,j}(r))
        
        \gamma_{i,j}(r) =  h_{i,j}(r) - c_{i,j}(r)
    
    '''
    def __init__(self):
        self.potential = None
        self.value = None
        
    def __repr__(self):
        return '<AtomicClosure: PercusYevick>'
    
    def calculate(self,gamma):
        
        assert self.potential is not None,'Potential for this closure is not set!'
        
        assert len(gamma) == len(self.potential),'Domain mismatch!'
        
        self.value = (np.exp(-self.potential)-1.0)*(1.0+gamma)
        
        return self.value
        
        

Overwriting ../typyPRISM/closure/PercusYevick.py


In [18]:
%%run_and_write ../typyPRISM/test/PercusYevick_TestCase.py
#!python
from __future__ import division,print_function
from typyPRISM.closure import PercusYevick
import unittest
import numpy as np

class PercusYevick_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a PercusYevick closure ?'''
        r = np.arange(0.75,3.5,0.05)
        U = np.zeros_like(r)
        U[r<1.0] = 1e6
        
        gamma = np.zeros_like(r)
        C1 = (np.exp(-U) - 1.0)*(1.0 + gamma)
        
        
        PY = PercusYevick()
        PY.potential = U
        C2 = PY.calculate(gamma)
        np.testing.assert_array_almost_equal(C1,C2)
        
        


Overwriting ../typyPRISM/test/PercusYevick_TestCase.py


## HypernettedChain

In [19]:
%%run_and_write ../typyPRISM/closure/HyperNettedChain.py
#!python
from __future__ import division,print_function
from typyPRISM.closure.AtomicClosure import AtomicClosure
import numpy as np
class HyperNettedChain(AtomicClosure):
    '''HyperNettedChain closure written in terms of a change of variables
    
    .. note::
    
        The change of variables is necessary in order to use potentials with
        hard cores. Written in the standard form, this closure diverges with
        divergent potentials, which makes it impossible to numerically solve. 
    
    .. math::
        
        c_{i,j}(r) = exp(\gamma_{i,j}(r)-U_{i,j}(r)) - 1.0 -  \gamma_{i,j}(r)
        
        \gamma_{i,j}(r) =  h_{i,j}(r) - c_{i,j}(r)
    
    '''
    def __init__(self):
        self.potential = None
        self.value = None
        
    def __repr__(self):
        return '<AtomicClosure: HyperNettedChain>'
    
    def calculate(self,gamma):
        
        assert self.potential is not None,'Potential for this closure is not set!'
        
        assert len(gamma) == len(self.potential),'Domain mismatch!'
        
        self.value = np.exp(gamma - self.potential) - 1.0 - gamma
        
        return self.value
        
        

Overwriting ../typyPRISM/closure/HyperNettedChain.py


In [20]:
%%run_and_write ../typyPRISM/test/HyperNettedChain_TestCase.py
#!python
from __future__ import division,print_function
from typyPRISM.closure import HyperNettedChain
import unittest
import numpy as np

class HyperNettedChain_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a HyperHyperNettedChain closure ?'''
        r = np.arange(0.75,3.5,0.05)
        U = np.zeros_like(r)
        U[r<1.0] = 1e6
        
        gamma = np.zeros_like(r)
        C1 = np.exp(gamma-U) - 1.0 - gamma
        
        
        HNC = HyperNettedChain()
        HNC.potential = U
        C2 = HNC.calculate(gamma)
        np.testing.assert_array_almost_equal(C1,C2)
        
        


Overwriting ../typyPRISM/test/HyperNettedChain_TestCase.py


## Molecular PercusYevick

In [46]:
%%run_and_write ../typyPRISM/closure/MolecularPercusYevick.py
#!python
from __future__ import division,print_function
from typyPRISM.closure.MolecularClosure import MolecularClosure
from typyPRISM.closure.PercusYevick import PercusYevick
from scipy.signal import fftconvolve
import numpy as np
class MolecularPercusYevick(MolecularClosure):
    '''Molecular Percus Yevick closure written in terms of a change of variables
    
    .. note::
    
        The change of variables is necessary in order to use potentials with
        hard cores. Written in the standard form, this closure diverges with
        divergent potentials, which makes it impossible to numerically solve. 
    
    .. math::
        
        c_{i,j}(r) = \omega(r) \star (exp(-U_{i,j}(r)) - 1.0) * (1.0 + \gamma_{i,j}(r)) \star \omega(r) \star
        
        \gamma_{i,j}(r) =  h_{i,j}(r) - c_{i,j}(r)
    
    '''
    def __init__(self):
        self._potential = None
        self.value = None
        self.atomic_closure = PercusYevick()
        
    @property
    def potential(self):
        return self._potential
    
    @potential.setter
    def potential(self,value):
        self._potential = value
        self.atomic_closure.potential = value
        
    def __repr__(self):
        return '<MolecularClosure: PercusYevick>'
    
    def calculate(self,gamma,omega1,omega2):
        
        assert self.potential is not None,'Potential for this closure is not set!'
        
        assert len(gamma) == len(self.potential),'Domain mismatch!'
        
        atomic_value = self.atomic_closure.calculate(gamma)
        
        # self.value = fftconvolve(fftconvolve(omega1,atomic_value,mode='same'),omega2,mode='same')
        self.value = np.convolve(np.convolve(omega1,atomic_value,mode='same'),omega2,mode='same')
        
        return self.value
        
        

Overwriting ../typyPRISM/closure/MolecularPercusYevick.py


In [47]:
%%run_and_write ../typyPRISM/test/MolecularPercusYevick_TestCase.py
#!python
from __future__ import division,print_function
from typyPRISM.closure import MolecularPercusYevick
from scipy.signal import fftconvolve
import unittest
import numpy as np

class MolecularPercusYevick_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a MolecularPercusYevick closure ?'''
        r = np.arange(0.75,3.5,0.05)
        U = np.zeros_like(r)
        U[r<1.0] = 1e6
        
        omega = np.ones_like(r)
        
        gamma = np.zeros_like(r)
        C1 = (np.exp(-U) - 1.0)*(1.0 + gamma)
        
        C1 = fftconvolve(fftconvolve(omega,C1,mode='same'),omega,mode='same')
        
        
        PY = MolecularPercusYevick()
        PY.potential = U
        C2 = PY.calculate(gamma,omega,omega)
        np.testing.assert_array_almost_equal(C1,C2)
        
        


Overwriting ../typyPRISM/test/MolecularPercusYevick_TestCase.py


## RUN ALL

In [48]:
import unittest
suite = []
suite.append(unittest.TestLoader().loadTestsFromTestCase(MolecularPercusYevick_TestCase))
suite.append(unittest.TestLoader().loadTestsFromTestCase(PercusYevick_TestCase))
suite.append(unittest.TestLoader().loadTestsFromTestCase(HyperNettedChain_TestCase))
suite = unittest.TestSuite(suite)
unittest.TextTestRunner(verbosity=2).run(suite)

test_create (__main__.MolecularPercusYevick_TestCase)
Can we create a MolecularPercusYevick closure ? ... ok
test_create (__main__.PercusYevick_TestCase)
Can we create a PercusYevick closure ? ... ok
test_create (__main__.HyperNettedChain_TestCase)
Can we create a HyperHyperNettedChain closure ? ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>